In [5]:
'''Text Classification Using Naive Bayes 

Step 1) Create Training set & Test set 

1.a) Manually create a small labeled dataset for a two-class problem (e.g. spam vs ham OR positive vs negative reviews).  Aim for 30 or more datapoints for this dataset.  Each datapoint consists of a tuple: a document and its class label.  The document can simply be a sentence or two.   

1.b) Split this dataset into Training set (80%) and Test set (20%). 

1.c) Store the created data into two files: train.csv and test.csv (One datapoint per line)

Step 2) Train a Naive Bayes classifier (Program 1)

2.a) Read in training set from the train.csv file 

2.b) Calculate the prior probabilities for both classes using the training set

2.b) Calculate the conditional probability of each unique word in the training set given a class. Compute these conditional probabilities for each class. 

2.c) Output the learnt classification model (all the above probabilities) to a file named model.csv

Step 3) Use the learnt NB classifier (Program 2)

3.a) Read in the model.csv file and the test.csv files 

3.b) Use the classification model to predict class label for each document in the test set

3.c) Output each of the test document, its predicted class label, and its actual class label to a file named test_predictions.csv

'''

import random
import re
import nltk
import pandas as pd

train = pd.read_csv('train.csv', index_col=0)
print(train.to_string())


class training:
    def print(self):
        print("hellow world")
        
    #reading the training data from csv file
    
    #2.b) Calculate the prior probabilities for both classes using the training set
    








def run():
    print("hello world")

if __name__ == "__main__":
    print("Enter \"q\" to end: ")
    print("Courpus: 1865-Lincoln.txt from inaugural corpus")
    run()


  Great quality, beautiful color.  We have had the same one for a few months now and love
1                                                                                        
1       Perfect. They do exactly what I need them to do. I will keep them for a long time
1     Excellent product and a much better quality than the one you get at Walmart for $50
0                                                          A little pricey for what it is
0                                             sheds all over, would not buy it ever again
1                           service was great, can't wait to try it out!Very good quality
0                              bottom will not stay on. zero customer service  do not buy
0                                                             Does not do a very good job
0                                                                        difficult to use
1                                                             I am happy with my purchase
0         

In [15]:
import random
import re
import nltk
import pandas as pd

train = pd.read_csv('train.csv', header=None, index_col=0)
#print(train.to_string())
print(train)

Int64Index([1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0,
            1, 0, 1, 1, 1, 1, 0, 1, 1, 1],
           dtype='int64', name=0)
